## Моделирование газовой динамики (распад произвольного разрыва)

In [95]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from tqdm import tqdm_notebook
from numba import jit
from scipy.optimize import root

In [96]:
@jit
def P(ro, u, e):
    """
    ro, u, e - scalars
    returns the value of P in this point 
    """
    return ro*(e-u**2/2)*(gamma-1)


@jit
def getE(ro, u, p):
    """
    ro, u, p - scalars
    returns the value of e in this point 
    """
    return p/(ro*(gamma-1))+u**2/2


@jit
def critical_velocities(p1, ro1, p2, ro2, gamma, u1, u2):
    """
    Calculate and print critical velocities, find configuration
    if P1 < P2
    """
    c1 = sqrt(gamma*p1/ro1)
    c2 = sqrt(gamma*p2/ro2)
    delta_u = u1-u2
    
    U_sh = (p2-p1)/sqrt(ro1*((gamma+1)*p2+(gamma+1)*p1)/2)
    U_out = -2*c2/(gamma-1)*(1-(p1/p2)**((gamma-1)/(2*gamma)))
    U_vac = -2*(c1+c2)/(gamma-1)
    
    print("u1 - u2 = ", delta_u)
    print("U_sh = ", U_sh)
    print("U_out = ", U_out)
    print("U_vac = ", U_vac)
    
    if delta_u >= U_sh:
        print("\nвправо и влево распространяется УВ")
    elif delta_u >= U_out:
        print("\nвлево распространяется УВ, вправо - ВР")
    elif delta_u >= U_vac:
        print("\nвправо и влево распространяются ВР")
    else:
        print("\nвправо и влево распространяются ВР, в центре образуется область вакуума")
    

def defineConfiguration(pL, p0, pR):
    if pL < p0:
        print('слева УВ')
    else:
        print('слева ВР')
        
    if pR < p0:
        print('\t\t\tсправа УВ')
    else:
        print('\t\t\tсправа ВР')    
    
    
@jit
def nextLayer(ro, u, e):
    """
    Метод крупных частиц
    ro, u, e - numpy arrays at previuos layer
    returns array with new numpy arrays
    """
    rv = ro.copy()
    uv = u.copy()
    ev = e.copy()
    
    new_ro = ro.copy()
    new_u = u.copy()
    new_e = e.copy()
    
    for i in range(1, N-1):
        pp = (P(ro[i], u[i], e[i]) + P(ro[i+1], u[i+1], e[i+1]))/2
        pn = (P(ro[i], u[i], e[i]) + P(ro[i-1], u[i-1], e[i-1]))/2
        
        up = (u[i] + u[i+1])/2
        un = (u[i] + u[i-1])/2
        
        uv[i] -= r*(pp-pn)/ro[i]
        ev[i] -= r*(pp*up-pn*un)/ro[i]
        
    for i in range(1, N-1):
        uvp = (uv[i] + uv[i+1])/2
        uvn = (uv[i] + uv[i-1])/2
        
        Dp = 0 if uvp >= 0 else 1
        Dn = 1 if uvn >= 0 else 0
        
        Yp = ro[i]*uvp if uvp >= 0 else -ro[i+1]*uvp
        Yn = ro[i-1]*uvn if uvn >= 0 else -ro[i]*uvn
        
        new_ro[i] = ro[i] +2*r*((Dn-0.5)*Yn + (Dp-0.5)*Yp)
        new_u[i] = uv[i]*ro[i]/new_ro[i] + r/new_ro[i]*(uv[i]*((Dn-1)*Yn+(Dp-1)*Yp)+uv[i-1]*Dn*Yn+uv[i+1]*Dp*Yp)
        new_e[i] = ev[i]*ro[i]/new_ro[i] + r/new_ro[i]*(ev[i]*((Dn-1)*Yn+(Dp-1)*Yp)+ev[i-1]*Dn*Yn+ev[i+1]*Dp*Yp)
        
    return [new_ro, new_u, new_e]


def F(ro, u, e, p):
    """
    returns F(f)
    """
    F = np.zeros((3, N))
    F[0] = ro*u
    F[1] = ro*u*u+p
    F[2] = u*(ro*e+p)
    
    return F


def smoothing(f):
    """
    f - np.array with shape (3, N)
    """
    new_f = f.copy()
    ro = f[0]
    for i in range(2, N-2):
        Dmm = ro[i-1] - ro[i-2]
        Dm = ro[i] - ro[i-1]
        
        Dp = ro[i+1] - ro[i]
        Dpp = ro[i+2] - ro[i+1]
         
        if (Dmm*Dm <= 0) or (Dm*Dp <= 0):
            Qm = f[:,i] - f[:,i-1]
        else:
            Qm = 0
            
        if (Dm*Dp <= 0) or (Dp*Dpp <= 0):
            Qp = f[:,i+1] - f[:,i]
        else:
            Qp = 0

        new_f[:,i] += q*(Qp - Qm)
        
    return new_f 
            

def nextLayer_MK(ro, u, e):
    """
    Метод Мак-Кормака
    ro, u, e - numpy arrays at previuos layer
    returns array with new numpy arrays
    """
    U = ro*u
    E = ro*e
    p = np.array(list(map(P, ro, u, e)))
    
    f = np.zeros((3, N))
    f[0] = ro
    f[1] = ro*u
    f[2] = ro*e
    
    Fn = F(ro, u, e, p)
    
    f_ = f.copy()
    new_f = f.copy()
    
    for i in range(N-1):
        f_[:,i] =  f[:,i] - r*(Fn[:,i+1] - Fn[:,i])
        
    p =  np.array(list(map(P, f_[0], f_[1]/ro, f_[2]/ro)))
    Fn_ = F(f_[0], f_[1]/f_[0], f_[2]/f_[0], p)
    
    for i in range(1, N):
        new_f[:,i] = (f[:,i]+f_[:,i])/2 - r/2 * (Fn_[:,i] - Fn_[:,i-1])
        
    new_f = smoothing(new_f)
        
    return [new_f[0], new_f[1]/new_f[0], new_f[2]/new_f[0]]

In [97]:
def a(ro, p, P):
    return sqrt(ro*((gamma+1)*P+(gamma-1)*p)/2)


def phi(p1, p2, u1, u2, a1, a2):
    return (a2*p1+a1*p2+a1*a2*(u1-u2))/(a1+a2)


def getP(ro1, ro2, p1, p2, u1, u2, iters = 100):
    P = (p1 + p2)/2
    for _ in range(iters):
        a1 = a(ro1, p1, P)
        a2 = a(ro2, p2, P)
        
        P = phi(p1, p2, u1, u2, a1, a2)
    U = (a1*u1+a2*u2+p1-p2)/(a1+a2)
    return [P, U]

In [98]:
def getc(ro, P):
    return sqrt(abs(gamma*P/ro))


def nonlinEq(x):
    """
    Система нелинейных уравнений для профиля
    https://ru.wikipedia.org/wiki/Задача_Римана_о_распаде_произвольного_разрыва
    """
    v2 = x[0]
    ro1 = x[1]
    ro2 = x[2]
    p = x[3]
    if p<0: p=0
    
    
    D = (p-PR)/(roR*v2)
    c2 = getc(ro2, p)
    cL = getc(roL, PL)
    
    a = 2*(cL-c2)/(gamma-1)
    b = roR*D/(D-v2)
    c = roL*(p/PL)**(1/gamma)
    d = PR*((gamma+1)*ro1 - (gamma-1)*roR)/((gamma+1)*roR - (gamma-1)*ro1)
    
    
    return [v2-a, ro1-b, ro2-c, p-d]

    
def nextLayer_ex(t):
    """
    Профиль точного решения
    t - current time
    returns array with new numpy arrays
    """    
    v2 = sol.x[0]
    ro1 = sol.x[1]
    ro2 = sol.x[2]
    P2 = sol.x[3]

    D = (P2-PR)/(roR*v2)
    c2 = getc(ro2, P2)
    cL = getc(roL, PL)
    
    ro = np.ones(N)
    u = np.ones(N)
    p = np.ones(N)
    
    ro[:N//2 + int(-cL*t/dx)] = roL
    u[:N//2 +int(-cL*t/dx)] = 0
    p[:N//2 +int(-cL*t/dx)] = PL
    
    for i in range(int(-cL*t/dx),int((v2-c2)*t/dx)):
        u_tmp = v2*(i*dx+cL*t)/((v2-c2+cL)*t)
        u[N//2+i] = u_tmp
        ro[N//2+i] = roL*(1-(gamma-1)/2*u_tmp/cL)**(2/(gamma-1))
        p[N//2+i] = PL*(1-(gamma-1)/2*u_tmp/cL)**(2*gamma/(gamma-1))
    
    ro[N//2 +int((v2-c2)*t/dx):N//2 +int(v2*t/dx)] = ro2
    u[N//2 +int((v2-c2)*t/dx):N//2 +int(v2*t/dx)] = v2
    p[N//2 +int((v2-c2)*t/dx):N//2 +int(v2*t/dx)] = P2
    
    ro[N//2 +int(v2*t/dx):N//2 +int(D*t/dx)] = ro1
    u[N//2 +int(v2*t/dx):N//2 +int(D*t/dx)] = v2
    p[N//2 +int(v2*t/dx):N//2 +int(D*t/dx)] = P2
    
    ro[N//2 +int(D*t/dx):] = roR
    u[N//2 +int(D*t/dx):] = 0
    p[N//2 +int(D*t/dx):] = PR
    
        
    return [ro, u, p]


def nextLayer_ex2(ro, u, p):
    """
    Профиль точного решения
    returns array with new numpy arrays
    """
    ro_ex = ro.copy()
    u_ex = u.copy()
    p_ex = np.array(p)
    
    left_pos = np.where(abs(u-u[N//2])<0.02)[0][0]
    right_pos = np.where(abs(u-u[N//2])<0.02)[0][-1]
    
    ro_ex[:left_pos] = ro[0]
    ro_ex[left_pos:right_pos+1] = ro[N//2]
    ro_ex[right_pos+1:] = ro[-1]
    
    u_ex[:left_pos] = u[0]
    u_ex[left_pos:right_pos+1] = u[N//2]
    u_ex[right_pos+1:] = u[-1]
    
    p_ex[:left_pos] = p[0]
    p_ex[left_pos:right_pos+1] = p[N//2]
    p_ex[right_pos+1:] = p[-1]    
    
    return [ro_ex, u_ex, p_ex]

In [99]:
dt = 0.0005
dx = 0.01

N = 600
T = 1000

r = dt/dx
gamma = 1.5

# best practise: [0.05, 0.2]
q = 0.1

ro = np.ones(N)
u = np.ones(N)
e = np.ones(N)

roL = 2.5
roR = 1.5

E = 1.5

PL = P(roL, 0, E)
PR = P(roR, 0, E)

ro[:N//2] = [roL]*(N//2)
ro[N//2:] = [roR]*(N//2)

u[:N//2] = [0]*(N//2)
u[N//2:] = [0]*(N//2)

e[:N//2] = [E]*(N//2)
e[N//2:] = [E]*(N//2)

ro_mk = ro.copy()
u_mk = u.copy()
e_mk = e.copy()

sol = root(nonlinEq, [0.2, 1.8, 2.2, 1.5])

# critical_velocities(p1 = P(ro[0], u[0], e[0]), ro1 = ro[0], p2 = P(ro[-1], u[-1], e[-1]), ro2 = ro[-1], 
#                     gamma = gamma, u1 = u[0], u2 = u[-1])


fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (11, 20))
xdata = dx*np.array(range(N)) - dx*N//2
ln1, = ax[0].plot([], [], color = 'b', linestyle = '--', label = 'Плотность (МКЧ)')
ln2, = ax[1].plot([], [], color = 'b', linestyle = '--', label = 'Скорость (МКЧ)')
ln3, = ax[2].plot([], [], color = 'b', linestyle = '--', label = 'Энергия (МКЧ)')
ln4, = ax[3].plot([], [], color = 'b', linestyle = '--', label = 'Давление (МКЧ)')

ln1_mk, = ax[0].plot([], [], color = 'r', linestyle = '--', label = 'Плотность (Мак-Кормака)')
ln2_mk, = ax[1].plot([], [], color = 'r', linestyle = '--', label = 'Скорость (Мак-Кормака)')
ln3_mk, = ax[2].plot([], [], color = 'r', linestyle = '--', label = 'Энергия (Мак-Кормака)')
ln4_mk, = ax[3].plot([], [], color = 'r', linestyle = '--', label = 'Давление (Мак-Кормака)')

ln1_ex, = ax[0].plot([], [], color = 'g', linestyle = '-', label = 'Плотность')
ln2_ex, = ax[1].plot([], [], color = 'g', linestyle = '-', label = 'Скорость')
ln3_ex, = ax[2].plot([], [], color = 'g', linestyle = '-', label = 'Энергия')
ln4_ex, = ax[3].plot([], [], color = 'g', linestyle = '-', label = 'Давление')


def init():
    for i in range(4):
        ax[i].set_xlim(xdata[0], xdata[-1])
        ax[i].grid(True)
        ax[i].legend()
    
    ax[0].set_ylim(1, 3)
    ax[1].set_ylim(0, 0.3)
    ax[2].set_ylim(1, 2)
    ax[3].set_ylim(1, 2)
    
    P_precise, U_precise = getP(ro1 = ro[0], ro2 = ro[-1], 
                     p1 = P(ro[0], u[0], e[0]), p2 = P(ro[-1], u[-1], e[-1]), 
                     u1 = u[0], u2 = u[-1],
                    iters = 1000)
    
    ax[3].hlines(P_precise, -10, 10, label = 'Точное давление на КР', color = 'y')
    ax[1].hlines(U_precise, -10, 10, label = 'Точная скорость на КР', color = 'y')
    
    defineConfiguration(PL, P_precise, PR)
    
    return ln1,


def update(t):
    global ro, u, e
    for _ in range(frame_const):
        try:
            ro, u, e = nextLayer(ro, u, e)
            p = list(map(P, ro, u, e))
        except:
            print(ro, u, e)
            raise Exception
    ln1.set_data(xdata, ro)
    ln2.set_data(xdata, u)
    ln3.set_data(xdata, e)
    ln4.set_data(xdata, p)
    
    global ro_mk, u_mk, e_mk
    for _ in range(frame_const):
        ro_mk, u_mk, e_mk = nextLayer_MK(ro_mk, u_mk, e_mk)
        p = list(map(P, ro_mk, u_mk, e_mk))
    ln1_mk.set_data(xdata, ro_mk)
    ln2_mk.set_data(xdata, u_mk)
    ln3_mk.set_data(xdata, e_mk)
    ln4_mk.set_data(xdata, p)
    
    global ro_ex, u_ex, e_ex
    for i in range(frame_const):
        ro_ex, u_ex, p_ex = nextLayer_ex(t*frame_const*dt+dt*i)
        e_ex = list(map(getE, ro_ex, u_ex, p_ex))
    ln1_ex.set_data(xdata, ro_ex)
    ln2_ex.set_data(xdata, u_ex)
    ln3_ex.set_data(xdata, e_ex)
    ln4_ex.set_data(xdata, p_ex)
    
    return ln1,


frame_const = 15
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=30, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

слева ВР
			справа УВ
слева ВР
			справа УВ


In [100]:
dt = 0.0005
dx = 0.01

N = 800
T = 1400

r = dt/dx
gamma = 1.5

# best practise: [0.05, 0.2]
q = 0.1

ro = np.ones(N)
u = np.ones(N)
e = np.ones(N)

roL = 1
roR = 1

E = 1.5

ro[:N//2] = [roL]*(N//2)
ro[N//2:] = [roR]*(N//2)

u[:N//2] = [1]*(N//2)
u[N//2:] = [-1]*(N//2)

e[:N//2] = [E]*(N//2)
e[N//2:] = [E]*(N//2)


ro_mk = ro.copy()
u_mk = u.copy()
e_mk = e.copy()

PL = P(roL, 1, E)
PR = P(roR, -1, E)

sol = root(nonlinEq, [-1, 1, 1, 1.5])

# critical_velocities(p1 = P(ro[0], u[0], e[0]), ro1 = ro[0], p2 = P(ro[-1], u[-1], e[-1]), ro2 = ro[-1], 
#                     gamma = gamma, u1 = u[0], u2 = u[-1])


fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (11, 20))
xdata = dx*np.array(range(N)) - dx*N//2
ln1, = ax[0].plot([], [], color = 'b', linestyle = '--', label = 'Плотность (МКЧ)')
ln2, = ax[1].plot([], [], color = 'b', linestyle = '--', label = 'Скорость (МКЧ)')
ln3, = ax[2].plot([], [], color = 'b', linestyle = '--', label = 'Энергия (МКЧ)')
ln4, = ax[3].plot([], [], color = 'b', linestyle = '--', label = 'Давление (МКЧ)')

ln1_mk, = ax[0].plot([], [], color = 'r', linestyle = '--', label = 'Плотность (Мак-Кормака)')
ln2_mk, = ax[1].plot([], [], color = 'r', linestyle = '--', label = 'Скорость (Мак-Кормака)')
ln3_mk, = ax[2].plot([], [], color = 'r', linestyle = '--', label = 'Энергия (Мак-Кормака)')
ln4_mk, = ax[3].plot([], [], color = 'r', linestyle = '--', label = 'Давление (Мак-Кормака)')


ln1_ex, = ax[0].plot([], [], color = 'g', linestyle = '-', label = 'Плотность')
ln2_ex, = ax[1].plot([], [], color = 'g', linestyle = '-', label = 'Скорость')
ln3_ex, = ax[2].plot([], [], color = 'g', linestyle = '-', label = 'Энергия')
ln4_ex, = ax[3].plot([], [], color = 'g', linestyle = '-', label = 'Давление')

def init():
    for i in range(4):
        ax[i].set_xlim(xdata[0], xdata[-1])
        ax[i].grid(True)
        ax[i].legend()
    
    ax[0].set_ylim(0, 5)
    ax[1].set_ylim(-1.5, 1.5)
    ax[2].set_ylim(1, 3)
    ax[3].set_ylim(0, 5)
    
    P_precise, U_precise = getP(ro1 = ro[0], ro2 = ro[-1], 
                     p1 = P(ro[0], u[0], e[0]), p2 = P(ro[-1], u[-1], e[-1]), 
                     u1 = u[0], u2 = u[-1],
                    iters = 1000)
    
    ax[3].hlines(P_precise, -10, 10, label = 'Точное давление на КР', color = 'y')
    ax[1].hlines(U_precise, -10, 10, label = 'Точная скорость на КР', color = 'y')
    
    defineConfiguration(PL, P_precise, PR)
    
    return ln1,


def update(t):
    global ro, u, e
    for _ in range(frame_const):
        ro, u, e = nextLayer(ro, u, e)
        p = list(map(P, ro, u, e))
    ln1.set_data(xdata, ro)
    ln2.set_data(xdata, u)
    ln3.set_data(xdata, e)
    ln4.set_data(xdata, p)
    
    global ro_mk, u_mk, e_mk
    for _ in range(frame_const):
        ro_mk, u_mk, e_mk = nextLayer_MK(ro_mk, u_mk, e_mk)
        p = list(map(P, ro_mk, u_mk, e_mk))
    ln1_mk.set_data(xdata, ro_mk)
    ln2_mk.set_data(xdata, u_mk)
    ln3_mk.set_data(xdata, e_mk)
    ln4_mk.set_data(xdata, p)
    
    global ro_ex, u_ex, e_ex
    ro_ex, u_ex, p_ex = nextLayer_ex2(ro_mk, u_mk, p)
    e_ex = list(map(getE, ro_ex, u_ex, p_ex))
    ln1_ex.set_data(xdata, ro_ex)
    ln2_ex.set_data(xdata, u_ex)
    ln3_ex.set_data(xdata, e_ex)
    ln4_ex.set_data(xdata, p_ex)
    
    return ln1,


frame_const = 30
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=50, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

C:\work\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in double_scalars


слева УВ
			справа УВ
слева УВ
			справа УВ


In [101]:
dt = 0.0005
dx = 0.01

N = 800
T = 2000

r = dt/dx
gamma = 1.5

# best practise: [0.05, 0.2]
q = 0.1

ro = np.ones(N)
u = np.ones(N)
e = np.ones(N)

ro[:N//2] = [1]*(N//2)
ro[N//2:] = [1.5]*(N//2)

u[:N//2] = [0]*(N//2)
u[N//2:] = [0]*(N//2)

e[:N//2] = [1.5]*(N//2)
e[N//2:] = [1.5]*(N//2)


roL = 1
roR = 1.5

E = 1.5

PL = P(roL, 0, E)
PR = P(roR, 0, E)

sol = root(nonlinEq, [0, 1, 1.5, 1.5])


ro_mk = ro.copy()
u_mk = u.copy()
e_mk = e.copy()

# critical_velocities(p1 = P(ro[0], u[0], e[0]), ro1 = ro[0], p2 = P(ro[-1], u[-1], e[-1]), ro2 = ro[-1], 
#                     gamma = gamma, u1 = u[0], u2 = u[-1])


fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (11, 20))
xdata = dx*np.array(range(N)) - dx*N//2
ln1, = ax[0].plot([], [], color = 'b', linestyle = '--', label = 'Плотность (МКЧ)')
ln2, = ax[1].plot([], [], color = 'b', linestyle = '--', label = 'Скорость (МКЧ)')
ln3, = ax[2].plot([], [], color = 'b', linestyle = '--', label = 'Энергия (МКЧ)')
ln4, = ax[3].plot([], [], color = 'b', linestyle = '--', label = 'Давление (МКЧ)')

ln1_mk, = ax[0].plot([], [], color = 'r', linestyle = '--', label = 'Плотность (Мак-Кормака)')
ln2_mk, = ax[1].plot([], [], color = 'r', linestyle = '--', label = 'Скорость (Мак-Кормака)')
ln3_mk, = ax[2].plot([], [], color = 'r', linestyle = '--', label = 'Энергия (Мак-Кормака)')
ln4_mk, = ax[3].plot([], [], color = 'r', linestyle = '--', label = 'Давление (Мак-Кормака)')


ln1_ex, = ax[0].plot([], [], color = 'g', linestyle = '-', label = 'Плотность')
ln2_ex, = ax[1].plot([], [], color = 'g', linestyle = '-', label = 'Скорость')
ln3_ex, = ax[2].plot([], [], color = 'g', linestyle = '-', label = 'Энергия')
ln4_ex, = ax[3].plot([], [], color = 'g', linestyle = '-', label = 'Давление')


def init():
    for i in range(4):
        ax[i].set_xlim(xdata[0], xdata[-1])
        ax[i].grid(True)
        ax[i].legend()
    
    ax[0].set_ylim(0.75, 1.75)
    ax[1].set_ylim(-0.5, 0.5)
    ax[2].set_ylim(1.25, 1.75)
    ax[3].set_ylim(0, 1.5)
    
    P_precise, U_precise = getP(ro1 = ro[0], ro2 = ro[-1], 
                     p1 = P(ro[0], u[0], e[0]), p2 = P(ro[-1], u[-1], e[-1]), 
                     u1 = u[0], u2 = u[-1],
                    iters = 1000)
    
    ax[3].hlines(P_precise, -10, 10, label = 'Точное давление на КР', color = 'y')
    ax[1].hlines(U_precise, -10, 10, label = 'Точная скорость на КР', color = 'y')
    
    defineConfiguration(PL, P_precise, PR)
    
    return ln1,


def update(t):
    global ro, u, e
    for _ in range(frame_const):
        ro, u, e = nextLayer(ro, u, e)
        p = list(map(P, ro, u, e))
    ln1.set_data(xdata, ro)
    ln2.set_data(xdata, u)
    ln3.set_data(xdata, e)
    ln4.set_data(xdata, p)
    
    global ro_mk, u_mk, e_mk
    for _ in range(frame_const):
        ro_mk, u_mk, e_mk = nextLayer_MK(ro_mk, u_mk, e_mk)
        p = list(map(P, ro_mk, u_mk, e_mk))
    ln1_mk.set_data(xdata, ro_mk)
    ln2_mk.set_data(xdata, u_mk)
    ln3_mk.set_data(xdata, e_mk)
    ln4_mk.set_data(xdata, p)
    
    global ro_ex, u_ex, e_ex
    ro_ex, u_ex, p_ex = nextLayer_ex2(ro_mk, u_mk, p)
    e_ex = list(map(getE, ro_ex, u_ex, p_ex))
    ln1_ex.set_data(xdata, ro_ex)
    ln2_ex.set_data(xdata, u_ex)
    ln3_ex.set_data(xdata, e_ex)
    ln4_ex.set_data(xdata, p_ex)
    
    return ln1,


frame_const = 30
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=50, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

C:\work\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
C:\work\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


слева УВ
			справа ВР
слева УВ
			справа ВР


In [102]:
dt = 0.0005
dx = 0.01

N = 600
T = 1000

r = dt/dx
gamma = 1.5

# best practise: [0.05, 0.2]
q = 0.1

ro = np.ones(N)
u = np.ones(N)
e = np.ones(N)

roL = 2.5
roR = 1.5

E = 1.5

PL = P(roL, 0, E)
PR = P(roR, 0, E)

ro[:N//2] = [roL]*(N//2)
ro[N//2:] = [roR]*(N//2)

u[:N//2] = [0]*(N//2)
u[N//2:] = [0]*(N//2)

e[:N//2] = [E]*(N//2)
e[N//2:] = [E]*(N//2)

ro_mk = ro.copy()
u_mk = u.copy()
e_mk = e.copy()

sol = root(nonlinEq, [0.2, 1.8, 2.2, 1.5])

# critical_velocities(p1 = P(ro[0], u[0], e[0]), ro1 = ro[0], p2 = P(ro[-1], u[-1], e[-1]), ro2 = ro[-1], 
#                     gamma = gamma, u1 = u[0], u2 = u[-1])


fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (11, 20))
xdata = dx*np.array(range(N)) - dx*N//2
ln1, = ax[0].plot([], [], color = 'b', linestyle = '--', label = 'Плотность (МКЧ)')
ln2, = ax[1].plot([], [], color = 'b', linestyle = '--', label = 'Скорость (МКЧ)')
ln3, = ax[2].plot([], [], color = 'b', linestyle = '--', label = 'Энергия (МКЧ)')
ln4, = ax[3].plot([], [], color = 'b', linestyle = '--', label = 'Давление (МКЧ)')

ln1_mk, = ax[0].plot([], [], color = 'r', linestyle = '--', label = 'Плотность (Мак-Кормака)')
ln2_mk, = ax[1].plot([], [], color = 'r', linestyle = '--', label = 'Скорость (Мак-Кормака)')
ln3_mk, = ax[2].plot([], [], color = 'r', linestyle = '--', label = 'Энергия (Мак-Кормака)')
ln4_mk, = ax[3].plot([], [], color = 'r', linestyle = '--', label = 'Давление (Мак-Кормака)')

ln1_ex, = ax[0].plot([], [], color = 'g', linestyle = '-', label = 'Плотность')
ln2_ex, = ax[1].plot([], [], color = 'g', linestyle = '-', label = 'Скорость')
ln3_ex, = ax[2].plot([], [], color = 'g', linestyle = '-', label = 'Энергия')
ln4_ex, = ax[3].plot([], [], color = 'g', linestyle = '-', label = 'Давление')


def init():
    for i in range(4):
        ax[i].set_xlim(xdata[0], xdata[-1])
        ax[i].grid(True)
        ax[i].legend()
    
    ax[0].set_ylim(1, 3)
    ax[1].set_ylim(0, 0.3)
    ax[2].set_ylim(1, 2)
    ax[3].set_ylim(1, 2)
    
    P_precise, U_precise = getP(ro1 = ro[0], ro2 = ro[-1], 
                     p1 = P(ro[0], u[0], e[0]), p2 = P(ro[-1], u[-1], e[-1]), 
                     u1 = u[0], u2 = u[-1],
                    iters = 1000)
    
    ax[3].hlines(P_precise, -10, 10, label = 'Точное давление на КР', color = 'y')
    ax[1].hlines(U_precise, -10, 10, label = 'Точная скорость на КР', color = 'y')
    
    defineConfiguration(PL, P_precise, PR)
    
    return ln1,


def update(t):
#     global ro, u, e
#     for _ in range(frame_const):
#         try:
#             ro, u, e = nextLayer(ro, u, e)
#             p = list(map(P, ro, u, e))
#         except:
#             print(ro, u, e)
#             raise Exception
#     ln1.set_data(xdata, ro)
#     ln2.set_data(xdata, u)
#     ln3.set_data(xdata, e)
#     ln4.set_data(xdata, p)
    
    global ro_mk, u_mk, e_mk
    for _ in range(frame_const):
        ro_mk, u_mk, e_mk = nextLayer_MK(ro_mk, u_mk, e_mk)
        p = list(map(P, ro_mk, u_mk, e_mk))
    ln1_mk.set_data(xdata, ro_mk)
    ln2_mk.set_data(xdata, u_mk)
    ln3_mk.set_data(xdata, e_mk)
    ln4_mk.set_data(xdata, p)
    
    global ro_ex, u_ex, e_ex
    ro_ex, u_ex, p_ex = nextLayer_ex2(ro_mk, u_mk, p)
    e_ex = list(map(getE, ro_ex, u_ex, p_ex))
    ln1_ex.set_data(xdata, ro_ex)
    ln2_ex.set_data(xdata, u_ex)
    ln3_ex.set_data(xdata, e_ex)
    ln4_ex.set_data(xdata, p_ex)
    
    return ln1,


frame_const = 15
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=30, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)

слева ВР
			справа УВ
слева ВР
			справа УВ
